In [1]:
import sys 
sys.path.append('../scripts/')
from dynamic_programming import *

/usr/local/lib/python3.10/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 2.0.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
class BeliefDynamicProgramming(DynamicProgramming):
    def __init__(self, widths, goal, puddles, time_interval, sampling_num, camera, puddle_coef=100.0, \
                 lowerleft=np.array([-4, -4]).T, upperright=np.array([4, 4]).T, dev_borders=[0.1,0.2,0.4,0.8]): 
        super().__init__(widths, goal, puddles, time_interval, sampling_num, puddle_coef, lowerleft, upperright)
        
 #       self.actions = [(0.0, 2.0), (0.0, -2.0), (1.0, 0.0), (-1.0, 0.0)]  #コメントアウト ###amdp7commentout
  #      self.state_transition_probs = self.init_state_transition_probs(time_interval, sampling_num)  #コメントアウト
        
        self.index_nums = np.array([*self.index_nums, len(dev_borders) + 1])
        nx, ny, nt, nh = self.index_nums
        self.indexes = list(itertools.product(range(nx), range(ny), range(nt), range(nh)))
        
        self.value_function, self.final_state_flags =  self.init_belief_value_function()
        self.policy = np.zeros(np.r_[self.index_nums,2]) 
        
        self.dev_borders = dev_borders
        self.dev_borders_side = [dev_borders[0]/10, *dev_borders, dev_borders[-1]*10]
        self.motion_sigma_transition_probs = self.init_motion_sigma_transition_probs()
        self.obs_sigma_transition_probs = self.init_obs_sigma_transition_probs(camera)
        
        self.depths = self.expected_depths(puddles) #__init__の最後に追加 ###amdp7expecteddepth
        
    def expected_depths(self, puddles, sampling_num=100): #追加
        depths = {}
        for index in itertools.product(range(self.index_nums[0]), range(self.index_nums[1]), range(self.index_nums[3])): #xyσのインデックス
            pose = self.pose_min[0:2] + self.widths[0:2]*(np.array(index[0:2]).T + 0.5) #セルの位置の中心
            sigma = (self.dev_borders_side[index[2]] + self.dev_borders_side[index[2]+1])/2 #σの中心
            belief = multivariate_normal(mean=pose, cov=np.eye(2)*(sigma**2)) #分布を構成（向きの次元は落としているので2次元）
            depth_sum = 0.0
            for pos in belief.rvs(sampling_num): #信念分布から位置をサンプリング
                depth_sum += sum([p.depth*p.inside(pos) for p in puddles])
                
            depths[index] = depth_sum/sampling_num
        return depths

    def init_obs_sigma_transition_probs(self, camera):
        probs = {}
        for index in self.indexes: 
            pose = self.pose_min + self.widths*(np.array(index[0:3]).T + 0.5) 
            sigma = (self.dev_borders_side[index[3]] + self.dev_borders_side[index[3]+1])/2
            S = (sigma**2)*np.eye(3) 
            
            for d in camera.data(pose):
                S = self.observation_update(d[1], S, camera, pose)
                    
            probs[index] = {self.cov_to_index(S):1.0}

        return probs
    
    def observation_update(self, landmark_id, S, camera, pose):
        distance_dev_rate = 0.14
        direction_dev = 0.05
        
        H = matH(pose, camera.map.landmarks[landmark_id].pos)
        estimated_z = IdealCamera.observation_function(pose, camera.map.landmarks[landmark_id].pos)
        Q = matQ(distance_dev_rate*estimated_z[0], direction_dev)
        K = S.dot(H.T).dot(np.linalg.inv(Q + H.dot(S).dot(H.T)))
        return (np.eye(3) - K.dot(H)).dot(S)
        
    def init_motion_sigma_transition_probs(self):
        probs = {}
        for a in self.actions:
            for i in range(len(self.dev_borders)+1):
                probs[(i, a)] = self.calc_motion_sigma_transition_probs(self.dev_borders_side[i], self.dev_borders_side[i+1], a)
                
        return probs
            
    def cov_to_index(self, cov):
        sigma = np.power(np.linalg.det(cov), 1.0/6)
        for i, e in enumerate(self.dev_borders):
            if sigma < e: return i
            
        return len(self.dev_borders)
        
    def calc_motion_sigma_transition_probs(self, min_sigma, max_sigma, action, sampling_num=100):
        nu, omega = action
        if abs(omega) < 1e-5: omega = 1e-5

        F = matF(nu, omega, self.time_interval, 0.0) #ロボットの向きは関係ないので0[deg]で固定で
        M = matM(nu, omega, self.time_interval, {"nn":0.19, "no":0.001, "on":0.13, "oo":0.2})#移動の誤差モデル（カルマンフィルタのものをコピペ）
        A = matA(nu, omega, self.time_interval, 0.0)
        
        ans = {}
        for sigma in np.linspace(min_sigma, max_sigma*0.999, sampling_num): #遷移前のσを作る（区間内に一様分布していると仮定）
            index_after = self.cov_to_index(sigma*sigma*F.dot(F.T) + A.dot(M).dot(A.T)) #遷移後のσのインデックス
            ans[index_after] = 1 if index_after not in ans else ans[index_after] + 1 #単にカウントしてるだけ（辞書の初期化もあるのでややこしい）
                
        for e in ans:
            ans[e] /= sampling_num #頻度を確率に

        return ans
    
    def init_belief_value_function(self): 
        v = np.empty(self.index_nums)
        f = np.zeros(self.index_nums) 
        
        for index in self.indexes:
            f[index] = self.belief_final_state(np.array(index).T)
            v[index] = self.goal.value if f[index] else -100.0
                
        return v, f
        
    def belief_final_state(self, index):
        x_min, y_min, _ = self.pose_min + self.widths*index[0:3] 
        x_max, y_max, _ = self.pose_min + self.widths*(index[0:3] + 1) 
        
        corners = [[x_min, y_min, _], [x_min, y_max, _], [x_max, y_min, _], [x_max, y_max, _] ] 
        return all([self.goal.inside(np.array(c).T) for c in corners ]) and index[3] == 0
    
    def action_value(self, action, index, out_penalty=True):###amdp7actionvalue
        value = 0.0
        for delta, prob in self.state_transition_probs[(action, index[2])]:
            after, out_reward = self.out_correction(np.array(index[0:3]).T + delta)
        
            for sigma_after, sigma_prob in self.motion_sigma_transition_probs[(index[3], action)].items():
                for sigma_obs, sigma_obs_prob in self.obs_sigma_transition_probs[(*after, sigma_after)].items():
                    reward = - self.time_interval * self.depths[(after[0], after[1], sigma_obs)] * self.puddle_coef \
                        - self.time_interval + out_reward*out_penalty #ここで報酬を計算。self.depthsのインデックスにsigma_obsを追加。
                    value += (self.value_function[(*after, sigma_obs)] + reward) * prob * sigma_prob * sigma_obs_prob

        return value

In [ ]:
puddles = [Puddle((-2, 0), (0, 2), 0.1), Puddle((-0.5, -2), (2.5, 1), 0.1)]

##地図とカメラを作る##
m = Map()
for ln in [(1,4), (4,1), (-4,-4)]: m.append_landmark(Landmark(*ln)) #位置をQMDPの実験に合わせる
c = IdealCamera(m)
    
dp = BeliefDynamicProgramming(np.array([0.2, 0.2, math.pi/18]).T, Goal(-3,-3), puddles, 0.1, 10, c) #カメラを加える

In [ ]:
def save():
    with open("policy_amdp.txt", "w") as f:  ###amdp6sweeps
        for index in dp.indexes:
            p = dp.policy[index]
            f.write("{} {} {} {} {} {}\n".format(index[0], index[1], index[2],index[3], p[0], p[1])) #一つ{}とindexの要素を増やす

    with open("value_amdp.txt", "w") as f:
        for index in dp.indexes:
            p = dp.value_function[index]
            f.write("{} {} {} {} {}\n".format(index[0], index[1], index[2], index[3], p)) #5行目と同じ

delta = 1e100
counter = 0

while delta > 0.01: 
    delta = dp.value_iteration_sweep()
    counter += 1
    print(counter, delta)
    save()

In [ ]:
import seaborn as sns
v = dp.value_function[:, :, 18, 4]
sns.heatmap(np.rot90(v), square=False)
plt.show()